In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random as random

In [2]:
def spin_lattice(N):
    spin = np.random.choice([-1,1], size=(N, N))

    return spin

In [3]:
def interaction_matrix_J(N, tt):

  matrix = 1.1*np.random.uniform(-1, 1, size=(len(tt), N, N, N, N))
  # matrix = np.ones([N,N,N,N])/20
  for t in range(len(tt)):
    for i in range(N):
      for j in range(N):
        matrix[t,i,j,i,j] = 0
  return matrix

interaction_matrix_J(5,[1,2,3]).shape

(3, 5, 5, 5, 5)

In [4]:
import random

def generate_indexes(N, p1, p2, p3):
    matrix = np.zeros((N, N), dtype=int)
    values = [1] * round(N**2 * p1) + [0] * round(N**2 * p2) + [-1] * round(N**2 * p3)

    random.shuffle(values)
    matrix.flat = values

    indexes = {"mean_reversion": [], "momentum": [], "value": []}
    for i in range(N):
      for j in range(N):
          if matrix[i,j] == 1:
            indexes["mean_reversion"].append([i,j])
          elif matrix[i,j] == 0:
            indexes["momentum"].append([i,j])
          elif matrix[i,j] == -1:
            indexes["value"].append([i,j])
    return indexes

generate_indexes(3,p1=0.3,p2=0.3,p3=0.4)

{'mean_reversion': [[0, 1], [1, 2], [2, 2]],
 'momentum': [[1, 1], [2, 0]],
 'value': [[0, 0], [0, 2], [1, 0], [2, 1]]}

In [5]:
def magnetic_field_h2(N, tt, t_evento, event_duration, tipo = "indst"):

    h_field_t0 = np.zeros([len(np.arange(tt[0],t_evento,1)),N,N])

    min_h_t = 0
    max_h_t = 0  

    if tipo == "pos":
      min_h_t = 0
      max_h_t = 0.5
    elif tipo == "neg":
      min_h_t = -0.5
      max_h_t = 0
    elif tipo == "indst": # indistinto, es decir, afecta a cada persona de manera aleatoria
      min_h_t = -0.5
      max_h_t = 0.5

    h_field_t_inh = np.random.uniform(min_h_t, max_h_t, size = (len(np.arange(t_evento,t_evento+event_duration,1)),N,N))

    h_field_t0_fin = np.zeros([len(np.arange(t_evento+event_duration,tt[-1]+1,1)),N,N])

    h_field_t = np.concatenate((h_field_t0,h_field_t_inh,h_field_t0_fin))

    return h_field_t

#magnetic_field_h(2,[0,1,2,3,4,5,6,7,8,9], 5, 2).shape

In [6]:
def magnetic_field_h(N, cumulative_magnetization, instant_magnetization, fair_price, t_indx, indexes, period=10):


    min_h_t = 0
    max_h_t = 0  

    h_field_t = np.zeros([N,N])

    # For mean_reversion and momentum:
    if t_indx > period:
      change = sum(instant_magnetization[t_indx-period:t_indx])
    else:
      change = sum(instant_magnetization[:t_indx])
      
    if change > 0:
      h_mean_reversion = np.random.uniform(-0.5,0)
      h_momentum = np.random.uniform(0,0.5)
    elif change < 0:
      h_mean_reversion = np.random.uniform(0,0.5)
      h_momentum = np.random.uniform(-0.5,0)
    else:
      h_mean_reversion = 0
      h_momentum = 0

    # For value:
    if cumulative_magnetization[t_indx-1] > fair_price:
      h_value = np.random.uniform(-0.5,0)
    elif cumulative_magnetization[t_indx-1] < fair_price:
      h_value = np.random.uniform(0,0.5)
    else: # If it is equal:
      h_value = 0


    for strategy, indxs in indexes.items():
      for i,j in indxs:
        if strategy == "mean_reversion":
          h_field_t[i,j] = h_mean_reversion
        elif strategy == "momentum":
          h_field_t[i,j] = h_momentum
        elif strategy == "value":
          h_field_t[i,j] = h_value


    return h_field_t

magnetic_field_h(3,[0,1,2,3,4,5,6,7,8,9], [0,-1,1,1,0,-1,1,-1,-1,1,0,1,1,1,-1,-1,0,1,-1,-1,1,-1,0], 4, 6, {'mean_reversion': [[0, 0], [0, 1], [1, 2]],
 'momentum': [[1, 0], [2, 0], [2, 2]],
 'value': [[0, 2], [1, 1], [2, 1]]})

array([[ 0.        ,  0.        , -0.35481787],
       [ 0.        , -0.35481787,  0.        ],
       [ 0.        , -0.35481787,  0.        ]])

In [7]:
def flip_energy(spin, interaction_matrix_J, matrix_h, i, j, t):
  E = 0
  E += 2*spin[i,j]*matrix_h[i,j]
  N = spin.shape[0]
  for k in range(N):
    for l in range(N):
      E += 2*interaction_matrix_J[t,i,j,k,l]*spin[i,j]*spin[k,l]

  return E

In [8]:
def active_agents(AA,N):
    
    x = random.sample(range(0, N), AA)
    y = random.sample(range(0, N), AA)

    return x, y


Backtest each strategy for 100 simulations

In [104]:
for i in range(0, 100, 1):
  N = 10
  AA = N
  t0 = 0
  tmax = 1000 # Un año
  TT = np.linspace(1,10,11)
  fair_price = 300
  p1,p2,p3 = 0.34, 0.33 , 0.33
  indexes = generate_indexes(N,p1,p2,p3)
  #print(indexes)

  event_time = 550
  event_duration = 200
  tipo_evento = "indst"

  h2 = magnetic_field_h2(N, tt, event_time, event_duration, tipo_evento) 

  tt = np.arange(t0,tmax + 1, 1)
  cum_mag = 100 # La ponemos a 100$
  spin = spin_lattice(N)
  J = interaction_matrix_J(N,tt)

  cum_mag_list = []
  inst_mag_list = []
  TT = [1]

  stocks_mr = 0
  cash_mr = 0
  stocks_momnt = 0
  cash_momnt = 0
  stocks_val = 0
  cash_val = 0

  agents_list = []

  for T in TT:
    for t in range(len(tt)):
        M_inst = 0
        if t == 0:
          h = np.zeros((N,N))
        else:
          h = magnetic_field_h(N,cum_mag_list,inst_mag_list,fair_price,t,indexes,period=10) + h2[t]
          #print(magnetic_field_h(N,cum_mag_list,inst_mag_list,fair_price,t,indexes,period=10).shape)
          #print(magnetic_field_h2(N, tt, event_time, event_duration, tipo_evento).shape)
          #print()
        # if t%100 == 0:
        #   print(t)
        #   print(h)
        AA_x, AA_y = active_agents(AA,N)
        for i in AA_x:
            for j in AA_y:

                flip_E = flip_energy(spin, J, h, i, j, t)
                uni_sample = random.uniform(0, 1)

                #print(flip_E)

                if flip_E <= 0:
                    spin[i, j] = -spin[i, j]

                elif (flip_E > 0) and (np.exp(-flip_E/T) >= uni_sample):
                    spin[i, j] = -spin[i, j]

                else:
                    spin[i, j] = spin[i, j]

                cum_mag += spin[i, j]
                M_inst += spin[i,j] 
        agents_list.append(spin)
        cum_mag_list.append(cum_mag)
        inst_mag_list.append(M_inst)

  for strategy, indxs in indexes.items():
      for t in range(1, len(tt), 1):
          for i,j in indxs:

              if strategy == "mean_reversion":
                  cash_mr -= agents_list[t][i][j]*cum_mag_list [t-1]
                  stocks_mr += agents_list[t][i][j]

              elif strategy == "momentum":
                  cash_momnt -= agents_list[t][i][j]*cum_mag_list [t-1]
                  stocks_momnt += agents_list[t][i][j]
                  
              elif strategy == "value":
                  cash_val -= agents_list[t][i][j]*cum_mag_list [t-1]
                  stocks_val += agents_list[t][i][j]

  traders_mr = len(indexes["mean_reversion"])
  traders_momnt = len(indexes["momentum"])
  traders_val = len(indexes["value"])

  if traders_mr == 0:
    a = 0
  else:
    a = 1/traders_mr

  if traders_momnt == 0:
    b = 0
  else:
    b = 1/traders_momnt

  if traders_val == 0:
    c = 0
  else:
    c = 1/traders_val


  profit = {"m_r": {"stocks": stocks_mr*a, "cash": cash_mr*a}, 
              "momnt": {"stocks": stocks_momnt*b, "cash": cash_momnt*b}, 
              "val": {"stocks": stocks_val*c, "cash": cash_val*c}}

  nw_mean_reversion = profit["m_r"]["cash"] + profit["m_r"]["stocks"]*cum_mag_list [-1]
  nw_momentum = profit["momnt"]["cash"] + profit["momnt"]["stocks"]*cum_mag_list [-1]
  nw_value = profit["val"]["cash"] + profit["val"]["stocks"]*cum_mag_list [-1]

  print (nw_mean_reversion, " ",nw_momentum, "  ", nw_value,)


6128.470588235294   5261.818181818184    13680.72727272728
2226.3529411764684   -5734.545454545456    -10322.181818181823
3911.294117647056   1007.454545454546    3022.3636363636324
-1698.1176470588252   1312.1818181818198    -437.39393939393904
-9586.352941176472   -14815.272727272726    24692.121212121212
1594.3529411764612   616.0    2258.666666666657
2624.705882352937   270.42424242424204    1892.969696969696
-26089.882352941175   -3360.060606060606    -16800.303030303032
-4260.352941176476   2194.7272727272757    6584.1818181818235
-3189.5294117647063   -547.69696969697    -8215.45454545453
-2100.235294117647   -1081.939393939394    -5409.696969696968
1685.4117647058847   434.121212121212    3907.0909090909117
-10368.117647058825   16023.454545454548    -26705.75757575758
-16912.23529411765   10164.42424242424    -4356.18181818182
7543.7647058823495   5829.272727272735    -1943.0909090909117
-17800.470588235286   -2292.484848484848    -2292.484848484848
2217.2941176470576   -3426.